In [1]:
import pandas as pd

data = pd.read_csv("agentic_output.csv")

data

,qid,pred
0,0133-3,**Observation:**\nThe video displays a compila...
1,0133-7,D
2,0142-0,The character in the Spiderman costume places ...
3,0142-1,Based on the video:\n\n1. **Observation:** Th...
4,0142-2,Based on the video:\n\n1. **Observation:** Th...
...,...,...
1495,1344-0,❓ **Not visually confirmable.**\n\nThe video s...
1496,1344-1,❓ **Not visually confirmable**\n\n1. **Observ...
1497,1344-2,❓ **Not visually confirmable**\n\n**Explanatio...
1498,1344-3,❓ **Not visually confirmable**\n\n**Explanatio...


In [64]:
import pandas as pd

data = pd.read_parquet("data/test-00000-of-00001.parquet")
data.head(15)

,qid,video_id,question_type,capability,question,duration,question_prompt,answer,youtube_url
0,0008-0,sj81PWrerDk,Primary Open-ended Question,Plot Attribute (Montage),What is the difference between the action of t...,8.85,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/sj81PWrerDk
1,0008-1,sj81PWrerDk,Paraphrased Open-ended Question,Plot Attribute (Montage),Can you describe how the actions of the last p...,8.85,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/sj81PWrerDk
2,0008-2,sj81PWrerDk,Correctly-led Open-ended Question,Plot Attribute (Montage),Did the last person open the bottle without us...,8.85,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/sj81PWrerDk
3,0008-3,sj81PWrerDk,Wrongly-led Open-ended Question,Plot Attribute (Montage),Did the last person in the video open the bott...,8.85,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/sj81PWrerDk
4,0008-7,sj81PWrerDk,Multiple-choice Question with a Single Correct...,Plot Attribute (Montage),How does the last person in the video open the...,8.85,E. None of the above\nSelect one best answer t...,,https://www.youtube.com/shorts/sj81PWrerDk
5,0012-0,AGCyLqLuUJ0,Primary Open-ended Question,Element Counting,How many robot figures appear in the video?,20.67,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/AGCyLqLuUJ0
6,0012-1,AGCyLqLuUJ0,Paraphrased Open-ended Question,Element Counting,What is the total number of robot figures show...,20.67,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/AGCyLqLuUJ0
7,0012-2,AGCyLqLuUJ0,Correctly-led Open-ended Question,Element Counting,Does the video display four robot figures in t...,20.67,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/AGCyLqLuUJ0
8,0012-3,AGCyLqLuUJ0,Wrongly-led Open-ended Question,Element Counting,Are there only two robot figures shown in the ...,20.67,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/AGCyLqLuUJ0
9,0012-7,AGCyLqLuUJ0,Multiple-choice Question with a Single Correct...,Element Counting,How many robot figures are shown in the video?...,20.67,E. None of the above\nSelect one best answer t...,,https://www.youtube.com/shorts/AGCyLqLuUJ0


In [65]:
qid = data['qid']
video_id = data['video_id']
question_type = data['question_type']
capability = data['capability']
question = data['question']
duration = data['duration']
question_prompt = data['question_prompt']
answer = data['answer']
youtube_url = data['youtube_url']

In [67]:
question_type.value_counts()

question_type
Primary Open-ended Question                              300
Paraphrased Open-ended Question                          300
Correctly-led Open-ended Question                        300
Wrongly-led Open-ended Question                          300
Multiple-choice Question with a Single Correct Answer    300
Name: count, dtype: int64

In [68]:
capability.value_counts()

capability
Element Counting                                           155
Objective Causality (Videography Phenomenon & Illusion)    130
Positional Relationship                                    120
Event Localization                                         110
Character Motivation Causality                             105
Element Attributes (Optical Illusion)                      100
Event Counting                                             100
Element Localization                                        90
Plot Attribute                                              90
Displacement Attribute                                      75
Professional Knowledge                                      70
Character Reaction Causality                                65
Character Emotion Attribute                                 60
Objective Causality                                         55
Element Attributes                                          55
Local Event Attribute                       

In [ ]:
from google import genai
from google.genai import types
from dotenv import load_dotenv
import pandas as pd
import os
import time

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

def generate(video_url, question, question_prompt):

    full_response = ""

    client = genai.Client(
        api_key=GOOGLE_API_KEY,
    )

    model = "gemini-2.5-pro-exp-03-25"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_uri(
                    file_uri=video_url,
                    mime_type="video/*",
                ),
                types.Part.from_text(text=question+question_prompt),
            ],
        ),
    ]

    system_instruction = """
    You are an AI that accurately analyzes the content of a given video. Your task is to evaluate whether a user's question correctly describes the events in the video.

    1. Watch the entire video carefully and understand the sequence of events.
    2. Analyze the user's question and compare it with what actually happens in the video.
    3. Provide a clear answer addressing the user's question based *only* on the visual evidence in the video. Explain your reasoning by referencing specific moments if necessary.
    """
    generate_content_config = types.GenerateContentConfig(
        temperature=0,
        response_mime_type="text/plain",
        system_instruction=system_instruction,
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        if chunk.text:
            print(chunk.text, end="")
            full_response += chunk.text
        print("\n----------------------")
    return full_response

all_responses = []

for i in range(3):
  id = test['qid'][i]
  all_responses.append(id)
  response = generate(video_url=test['youtube_url'][i], question=test['question'][i], question_prompt=test['question_prompt'][i])
  all_responses.append(response)
  time.sleep(12)

Okay, here's the analysis of the actions
----------------------
 in the video:

The first two people in the video tap the cap/neck of their respective bottles with a small object, seemingly attempting to loosen or remove the cap.

The last person initially taps the cap/neck of the Coca-Cola bottle similarly, but then performs a different action. Instead of removing the cap, they
----------------------
 strike the top of the bottle with the side of their hand, causing the *bottom* of the bottle to break off cleanly, leaving the cap and the top portion intact momentarily.

**In summary:** The first two people attempt to open the bottle at the cap, while the last person performs a trick that breaks the bottom
----------------------
 off the bottle.
----------------------
Yes, the actions of the last person in the video differ
----------------------
 significantly from the other individuals.

*   The first two people successfully open their bottles by tapping the cap, causing it to pop off

In [61]:
all_responses

['0008-0',
 "Okay, here's the analysis of the actions in the video:\n\nThe first two people in the video tap the cap/neck of their respective bottles with a small object, seemingly attempting to loosen or remove the cap.\n\nThe last person initially taps the cap/neck of the Coca-Cola bottle similarly, but then performs a different action. Instead of removing the cap, they strike the top of the bottle with the side of their hand, causing the *bottom* of the bottle to break off cleanly, leaving the cap and the top portion intact momentarily.\n\n**In summary:** The first two people attempt to open the bottle at the cap, while the last person performs a trick that breaks the bottom off the bottle.",
 '0008-1',
 "Yes, the actions of the last person in the video differ significantly from the other individuals.\n\n*   The first two people successfully open their bottles by tapping the cap, causing it to pop off while leaving the bottle intact.\n*   The last person attempts a similar tapping m

In [62]:
reshaped_data = [[all_responses[i], all_responses[i+1]] for i in range(0, len(all_responses), 2)]

df = pd.DataFrame(reshaped_data, columns=['qid', 'pred'])

In [63]:
df

,qid,pred
0,0008-0,"Okay, here's the analysis of the actions in th..."
1,0008-1,"Yes, the actions of the last person in the vid..."
2,0008-2,"Yes, the last person in the video opened the b..."
